In [14]:
import numpy as np
import pandas as pd
from fastai.text.all import *
import re

In [2]:
origin = pd.read_csv('train.csv')
origin.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


# Data Processing

In [3]:
def replace_enter(item):
    return item.replace('\n', ' ')

origin['excerpt'] = origin['excerpt'].apply(replace_enter)

In [4]:
origin['license'] = origin['license'].fillna(' ')

In [5]:
origin['url_legal'] = origin['url_legal'].fillna(' ')

In [6]:
def remove_host(item):
    item = item.strip()\
            .replace('https://simple.wikipedia.org/wiki/', '')\
            .replace('https://en.wikibooks.org/wiki/', '')\
            .replace('https://en.wikipedia.org/wiki/', '')\
            .replace('http://static.ehe.osu.edu/sites/beyond/penguins/downloads/feature-stories/','')\
            .replace('https://www.ck12.org/book/', '')\
            .replace('https://www.commonlit.org/texts/', '')\
            .replace('https://emedia.uen.org/courses/', '')\
            .replace('http://ukuqonda.co.za/digicom/', '')\
            .replace('https://sites.ehe.osu.edu/beyondpenguins/files/2011/06/', '')\
            .replace('https://beyondpenguins.ehe.osu.edu/issue/a-sense-of-place/', '')\
            .replace('https://drive.google.com/file/d/0B6pcGxi_-Y9PdWJhVXVrVnIzVnM/view', 'Grade-7A')\
            .replace('https://drive.google.com/file/d/0B6pcGxi_-Y9PRzdONUY2a1Y5aTg/view', 'Grade-8A')\
            .replace('https://drive.google.com/file/d/0Bz69EszGvRvaYnlFU1Zfb0hHNXc/view', 'Grade-11A')\
            .replace('.pdf', '')
            

    item = re.sub(r'https://www.digitallibrary.io/en/books/.+', 'Digital', item)
    item = re.sub(r'.+kid.+', 'Kid', item)
    item = re.sub(r'.+kid.+', 'Kid', item)
    item = re.sub(r'https://www.africanstorybook.org.+', 'African', item)
    return item.replace('/', ' ')\
            .replace('_', ' ')\
            .replace('-', ' ')

origin['url_legal'] = origin['url_legal'].apply(remove_host)

In [7]:
origin['text'] = origin['license'] + ' ' + origin['url_legal'] + ' ' + origin['excerpt']

In [8]:
df = origin[['text', 'target']]

# Tokenizer

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [35]:
train_tensor = tokenizer(list(df["text"]), padding="max_length", 
                         truncation=True, max_length=30, 
                         return_tensors="pt")["input_ids"]

In [36]:
class Dataset:
    def __init__(self, tensors, targ, ids):
        self.text = tensors[ids]
        self.targ = targ[ids].reset_index(drop=True)
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, idx):
        t = self.text[idx]
        y = self.targ[idx]
        return t, tensor(y)

In [40]:
# Split the data in a trainig and validation set
train_ids, valid_ids = RandomSplitter()(df)

# Separate the y / target into a variable
target = df["target"]

# create the input dataset based on the randomsplitter ids and utiliing the pre-processed tokens 
train_ds = Dataset(train_tensor, target, train_ids)
valid_ds = Dataset(train_tensor, target, valid_ids)

train_dl = DataLoader(train_ds, bs=64)
valid_dl = DataLoader(valid_ds, bs=512)

dls = DataLoaders(train_dl, valid_dl)

In [41]:
bert = AutoModelForSequenceClassification.from_pretrained("bert-base-cased").train()

regressor = nn.Sequential(
    nn.ReLU(),
    nn.Linear(768,512),
    nn.Linear(512,256),
    nn.Linear(256, 1)
)

class BertRegressor(Module):
    def __init__(self, bert):
        self.bert = bert
    def forward(self, x):
        return self.bert.forward(x)

model = BertRegressor(regressor)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [42]:
learn = Learner(dls, model, loss_func=nn.CrossEntropyLoss(), metrics=[mse])
learn.lr_find()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (768x512 and 768x512)